<a href="https://colab.research.google.com/github/jbloewencolon/Creating-Dataset-for-The-Demographics-of-Faerun/blob/main/DnD_Dataset_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
# File path
df_file_path = '/content/drive/MyDrive/Colab Notebooks/DnD/fictional_data.xlsx'
idf_file_path = '/content/drive/MyDrive/Colab Notebooks/DnD/idf_dataset.csv'

# Export DataFrame to Excel
df.to_excel(df_file_path, index=False)

# Export the idf dataset to a CSV file
#idf.to_csv('/content/drive/MyDrive/Colab Notebooks/DnD/idf_dataset.csv', index=False)

print(f'DataFrame exported to: {df_file_path}')

DataFrame exported to: /content/drive/MyDrive/Colab Notebooks/DnD/fictional_data.xlsx


In [ ]:
# Read the Excel file into a DataFrame
df = pd.read_excel(df_file_path)

# Read the Excel file into a DataFrame
idf = pd.read_csv(idf_file_path)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648 entries, 0 to 647
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   settlement                  648 non-null    object 
 1   region_kingdom              648 non-null    object 
 2   capital                     648 non-null    object 
 3   settlement_population       648 non-null    int64  
 4   area                        365 non-null    float64
 5   ruler                       536 non-null    object 
 6   military                    622 non-null    object 
 7   languages                   646 non-null    object 
 8   religions                   645 non-null    object 
 9   exports                     647 non-null    object 
 10  imports                     623 non-null    object 
 11  population_breakdown        648 non-null    object 
 12  sources                     538 non-null    object 
 13  region_population           648 non

In [ ]:
df.head()

,settlement,region_kingdom,capital,settlement_population,area,ruler,military,languages,religions,exports,...,region_population,settlement_economy,class_density,demographic_breakdown,average_age,rumored_treasure_value,government_type,most_likely_cause_of_death,tax_rate,region_economy
0,Athkatla,Amn,Athkatla,116163,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,2920094,42145455,"{'Aristocratic': 15, 'Wealthy': 3644, 'Comfort...","{'humans': 116046, 'halflings': 116, 'half-orc...",39,858487,Confederacy,Starvation,19,135421205
1,Amnwater,Amn,Athkatla,7035,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,2920094,2907590,"{'Aristocratic': 1, 'Wealthy': 122, 'Comfortab...","{'humans': 2335, 'halflings': 4699, 'half-orcs...",55,300,Matriarchy,Starvation,0,135421205
2,Citadel Amnur,Amn,Athkatla,636,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,2920094,115340,"{'Aristocratic': 0, 'Wealthy': 3, 'Comfortable...","{'humans': 580, 'halflings': 55, 'half-orcs': ...",43,1,Patriarchy,Poisonous plants,0,135421205
3,Citadel Rashturl,Amn,Athkatla,710,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,2920094,61320,"{'Aristocratic': 0, 'Wealthy': 6, 'Comfortable...","{'humans': 487, 'halflings': 222, 'half-orcs':...",49,1,Monarchy,Exposure to the elements,0,135421205
4,Coryllvol,Amn,Athkatla,23000,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,2920094,8220165,"{'Aristocratic': 93, 'Wealthy': 665, 'Comforta...","{'humans': 5842, 'halflings': 17158, 'half-orc...",58,6684,Kleptocracy,Exposure to the elements,16,135421205


In [ ]:
lifestyle_costs = {
    'Wretched': 0,          # No cost
    'Squalid': 1,           # 1 silver piece (sp) per day
    'Poor': 2,              # 2 silver pieces (sp) per day
    'Modest': 1,            # 1 gold piece (gp) per day
    'Comfortable': 2,       # 2 gold pieces (gp) per day
    'Wealthy': 4,           # 4 gold pieces (gp) per day
    'Aristocratic': 10      # 10 gold pieces (gp) minimum per day
}

# Define lifestyle ranges and corresponding lifestyle names for different population thresholds
lifestyle_ranges = {
    'tiny_settlement': [
        (0, 0.001),     # Aristocratic: .5%
        (0, 1),    # Wealthy: 1%
        (1, 40),   # Comfortable: 39%
        (40, 99),   # Modest: 59%
        (99, 100),   # Squalid: 1%
    ],
    'small_settlement': [
        (0.00001, .05),     # Aristocratic: .5%
        (.05, 5),    # Wealthy: 5.95%
        (5, 30),   # Comfortable: 25%
        (30, 80),   # Modest: 60%
        (80, 98),   # Squalid: 18%
        (98, 100)   # Wretched: 2%
    ],
    'medium_settlement': [
        (0.001, 1),     # Aristocratic: 1%
        (3, 13),    # Wealthy: 12%
        (13, 45),   # Comfortable: 32%
        (45, 84),   # Modest: 39%
        (84, 96),   # Squalid: 12%
        (96, 100)   # Wretched: 4%
    ],
    'large_settlement': [
        (0.5, 1.3),     # Aristocratic: 1%
        (3, 15),    # Wealthy: 13%
        (15, 48),   # Comfortable: 30%
        (48, 78),   # Modest: 30%
        (78, 93),   # Squalid: 15%
        (93, 100)   # Wretched: 7%
    ],
    'mega_settlement': [
        (1, 2),     # Aristocratic: 2%
        (3, 15),    # Wealthy: 13%
        (15, 48),   # Comfortable: 30%
        (48, 78),   # Modest: 30%
        (78, 93),   # Squalid: 15%
        (93, 100)   # Wretched: 7%
    ]
}


In [ ]:
lifestyle_costs = {
    'Wretched': 0,
    'Squalid': 1,
    'Poor': 2,
    'Modest': 1,
    'Comfortable': 2,
    'Wealthy': 4,
    'Aristocratic': 10
}

lifestyle_ranges = {
    'tiny_settlement': {'Aristocratic': (0, 0), 'Wealthy': (0, 1), 'Comfortable': (1, 40), 'Modest': (1, 50), 'Squalid': (0, 9)},
    'small_settlement': {'Aristocratic': (0, .05), 'Wealthy': (.05, 5), 'Comfortable': (5, 30), 'Modest': (30, 55), 'Squalid': (10, 18), 'Wretched': (1, 2)},
    'medium_settlement': {'Aristocratic': (0, 1), 'Wealthy': (1, 3), 'Comfortable': (10, 33), 'Modest': (13, 35), 'Squalid': (10, 20), 'Wretched': (3, 12)},
    'large_settlement': {'Aristocratic': (0, 1), 'Wealthy': (1, 5), 'Comfortable': (15, 25), 'Modest': (15, 38), 'Squalid': (12, 20), 'Wretched': (8, 15)},
    'mega_settlement': {'Aristocratic': (0, 2), 'Wealthy': (2, 7), 'Comfortable': (18, 35), 'Modest': (15, 38), 'Squalid': (15, 25), 'Wretched': (10, 15)}
}

def assign_lifestyle_demographics(population):
    total_economy = 0
    if population < 1000:
        city_size = 'tiny_settlement'
    elif population <= 10000:
        city_size = 'small_settlement'
    elif population <= 100000:
        city_size = 'medium_settlement'
    elif population <= 1000000:
        city_size = 'large_settlement'
    else:
        city_size = 'mega_settlement'

    class_density = {}
    for lifestyle, (start, end) in lifestyle_ranges[city_size].items():
        percentage = random.uniform(start, end)
        count = int(population * (percentage / 100))
        class_density[lifestyle] = count
        total_economy += count * lifestyle_costs[lifestyle] * 365

    return class_density, total_economy

In [ ]:
df['class_density'], df['settlement_economy'] = zip(*df['settlement_population'].apply(assign_lifestyle_demographics))

# Assuming you have a DataFrame 'df' with columns containing float values
columns_to_convert = ['settlement_population', 'region_population', 'settlement_economy', 'average_age', 'rumored_treasure_value', 'region_economy']

# Fill NaN values with 0 and convert infinity values to a large integer
for column in columns_to_convert:
    df[column] = df[column].fillna(0).replace([np.inf, -np.inf], np.nan).astype(int)

# Use astype(int) to convert specified columns to integers
df[columns_to_convert] = df[columns_to_convert].astype(int)

In [ ]:
df.head()

,settlement,region_kingdom,capital,settlement_population,area,ruler,military,languages,religions,exports,...,settlement_economy,class_density,demographic_breakdown,average_age,rumored_treasure_value,government_type,most_likely_cause_of_death,tax_rate,hidden_economy,region_economy
0,Athkatla,Amn,Athkatla,116163,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,42145455,"{'Aristocratic': 15, 'Wealthy': 3644, 'Comfort...","{'humans': 116046, 'halflings': 116, 'half-orc...",39,80378,Confederacy,Starvation,19,271797,135421205
1,Amnwater,Amn,Athkatla,7035,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,2907590,"{'Aristocratic': 1, 'Wealthy': 122, 'Comfortab...","{'humans': 2335, 'halflings': 4699, 'half-orcs...",55,3,Matriarchy,Starvation,0,2,135421205
2,Citadel Amnur,Amn,Athkatla,636,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,115340,"{'Aristocratic': 0, 'Wealthy': 3, 'Comfortable...","{'humans': 580, 'halflings': 55, 'half-orcs': ...",43,0,Patriarchy,Poisonous plants,0,0,135421205
3,Citadel Rashturl,Amn,Athkatla,710,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,61320,"{'Aristocratic': 0, 'Wealthy': 6, 'Comfortable...","{'humans': 487, 'halflings': 222, 'half-orcs':...",49,0,Monarchy,Exposure to the elements,0,0,135421205
4,Coryllvol,Amn,Athkatla,23000,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,8220165,"{'Aristocratic': 93, 'Wealthy': 665, 'Comforta...","{'humans': 5842, 'halflings': 17158, 'half-orc...",58,134,Kleptocracy,Exposure to the elements,16,890,135421205


In [ ]:
# Calculate the total economy across all settlements
total_economy = df['settlement_economy'].sum()
print(f"Total Economy: {total_economy}")

# Find the top 10 settlements with the highest settlement economy
top_10_settlements = df.nlargest(10, 'settlement_economy')

# Print the top 10 settlements
print("Top 10 Settlements by Economy:")
print(top_10_settlements[['settlement', 'settlement_economy', 'settlement_population']])


Total Economy: 2536637945
Top 10 Settlements by Economy:
        settlement  settlement_economy  settlement_population
450      Waterdeep           664490165                1347840
622          Skuld            71829810                 193245
625     Gheldaneth            66307725                 170413
441      Unthalass            62303675                 158047
59          Suzail            51553330                 129311
456  Baldur's Gate            51319000                 125000
362      Zazesspur            43526980                 114508
0         Athkatla            42145455                 116163
31       Calimport            34823920                  86268
306       Darromar            25708775                  67416


In [ ]:
# Group by 'Region' and sum up the 'settlement_economy'
region_economy_sum = df.groupby('region_kingdom')['settlement_economy'].sum().reset_index()

# Rename the columns for easier merging
region_economy_sum.columns = ['region_kingdom', 'new_region_economy']

# Merge the DataFrame to include the new region economy sums
df = pd.merge(df, region_economy_sum, on='region_kingdom', how='left')

# Update the 'region_economy' column with the newly computed sums
df['region_economy'] = df['new_region_economy']

# Drop the temporary column used for merging
df.drop(columns=['new_region_economy'], inplace=True)


In [ ]:
def determine_settlement_size(population):
    if population < 1000:
        return "tiny"
    elif population <= 10000:
        return "small"
    elif population <= 100000:
        return "medium"
    elif population <= 1000000:
        return "large"
    else:
        return "mega"

In [ ]:
def adjust_settlement_economy(row):
    if row['settlement_size'] == 'tiny':
        return row['settlement_economy'] / 10000
    elif row['settlement_size'] == 'small':
        return row['settlement_economy'] / 1000
    elif row['settlement_size'] == 'medium':
        return row['settlement_economy'] / 100
    elif row['settlement_size'] == 'large':
        return row['settlement_economy'] / 10
    else:
        return row['settlement_economy']

# Apply 'determine_settlement_size' to create a 'settlement_size' column temporarily
idf['settlement_size'] = idf['settlement_population'].apply(determine_settlement_size)

# Save this column as a separate Series for future use
settlement_size_series = idf['settlement_size'].copy()

# Adjust the 'settlement_economy' column
idf['settlement_economy'] = df.apply(adjust_settlement_economy, axis=1)

# Drop the 'settlement_size' column since you want to save the DataFrame without it
idf.drop(columns=['settlement_size'], inplace=True)

KeyError: ignored

In [ ]:
# Assuming settlement_economy is already calculated
def calculate_hidden_economy(row):
    try:
        class_density = ast.literal_eval(row['class_density'])
        squalid_population = class_density.get('Squalid', 0)
        squalid_multiplier = squalid_population * 0.001
        wealthy_population = class_density.get('Wealthy', 0)
        wealthy_multiplier = wealthy_population * 0.01
        aristocratic_population = class_density.get('Aristocratic', 0)
        aristocratic_multiplier = aristocratic_population * 1.5
        hidden_multiplier = wealthy_multiplier + aristocratic_multiplier + squalid_multiplier
        hidden_economy = row['settlement_economy'] / 20000 * hidden_multiplier if hidden_multiplier != 0 else row['settlement_economy']
        return hidden_economy
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Reset 'hidden_economy' column to all 0s in idf
idf['hidden_economy'] = 0

# Recalculate 'hidden_economy' using the updated function
idf['hidden_economy'] = idf.apply(calculate_hidden_economy, axis=1)

# Convert the 'hidden_economy' column to integers, handling None values
idf['hidden_economy'] = idf['hidden_economy'].fillna(0).astype(int)

# Calculate hidden economy per capita and print the top 10 settlements
idf['hidden_economy_per_capita'] = idf['hidden_economy'] / idf['settlement_population']

# Get the top 10 settlements with the highest hidden economy per capita
top_10_hidden_economies = idf.nlargest(10, 'hidden_economy_per_capita')[['settlement', 'hidden_economy_per_capita']]

# Print the top 10 hidden economies per capita
print("Top 10 Settlements with Highest Hidden Economy Per Capita:")
print(top_10_hidden_economies)

Top 10 Settlements with Highest Hidden Economy Per Capita:
        settlement  hidden_economy_per_capita
450      Waterdeep                 426.504656
59          Suzail                  43.058433
456  Baldur's Gate                  41.475384
441      Unthalass                  39.359779
622          Skuld                  39.105969
0         Athkatla                  36.768024
625     Gheldaneth                  35.293921
362      Zazesspur                  34.601809
459        Mirabar                  32.831301
581         Dorset                   4.860761


In [ ]:
df['hidden_economy'].head()

0    271797
1         2
2         0
3         0
4       890
Name: hidden_economy, dtype: int64

In [ ]:
df['settlement_economy'] = df['settlement_economy'].astype(int)

# Calculate the total economy of all settlements
total_economy = df['settlement_economy'].sum() + df['hidden_economy'].sum()

print(f"The total economy of all settlements is {total_economy}")


The total economy of all settlements is 3360065401


In [ ]:
# Function to adjust hidden economy based on the number of unique imports and exports
def adjust_hidden_economy(row):
    # Get the number of unique exports and imports
    unique_exports = len(set(row['exports'])) if isinstance(row['exports'], list) else 0
    unique_imports = len(set(row['imports'])) if isinstance(row['imports'], list) else 0

    # Calculate adjustment value
    adjustment_value = (unique_exports + unique_imports) / 100 * row['settlement_economy']

    # Update hidden_economy value
    return row['hidden_economy'] + adjustment_value

# Assuming 'hidden_economy' is already initialized. If not, initialize it first.
if 'hidden_economy' not in idf.columns:
    idf['hidden_economy'] = 0.0  # Initialize to 0

# Now adjust the 'hidden_economy' column
idf['hidden_economy'] = idf.apply(adjust_hidden_economy, axis=1)
idf['hidden_economy'] = idf['hidden_economy'].astype(int)

In [ ]:
import ast  # For parsing the string representation of a dictionary

# Custom function to calculate demographic_breakdown
def calculate_demographics(row):
    try:
        # Parse the population_breakdown if it's a string representation of a dictionary
        if isinstance(row['population_breakdown'], str):
            population_breakdown = ast.literal_eval(row['population_breakdown'])
        else:
            population_breakdown = row['population_breakdown']

        # Initialize the demographic_breakdown dictionary
        demographic_breakdown = {}

        # Calculate absolute populations based on percentages
        for species, percentage in population_breakdown.items():
            demographic_breakdown[species] = int(row['settlement_population'] * (percentage / 100))

        return demographic_breakdown
    except Exception as e:
        print(f"Error calculating demographics for row: {e}")
        return None

# Apply the function across the DataFrame rows
df['demographic_breakdown'] = df.apply(calculate_demographics, axis=1)


In [ ]:
df.head(10)

,settlement,region_kingdom,capital,settlement_population,area,ruler,military,languages,religions,exports,...,settlement_economy,class_density,demographic_breakdown,average_age,rumored_treasure_value,government_type,most_likely_cause_of_death,tax_rate,hidden_economy,region_economy
0,Athkatla,Amn,Athkatla,116163,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,42145455,"{'Aristocratic': 15, 'Wealthy': 3644, 'Comfort...","{'humans': 116046, 'halflings': 116, 'half-orc...",39,80378,Confederacy,Starvation,19,271797,135421205
1,Amnwater,Amn,Athkatla,7035,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,2907590,"{'Aristocratic': 1, 'Wealthy': 122, 'Comfortab...","{'humans': 2335, 'halflings': 4699, 'half-orcs...",55,3,Matriarchy,Starvation,0,2,135421205
2,Citadel Amnur,Amn,Athkatla,636,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,115340,"{'Aristocratic': 0, 'Wealthy': 3, 'Comfortable...","{'humans': 580, 'halflings': 55, 'half-orcs': ...",43,0,Patriarchy,Poisonous plants,0,0,135421205
3,Citadel Rashturl,Amn,Athkatla,710,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,61320,"{'Aristocratic': 0, 'Wealthy': 6, 'Comfortable...","{'humans': 487, 'halflings': 222, 'half-orcs':...",49,0,Monarchy,Exposure to the elements,0,0,135421205
4,Coryllvol,Amn,Athkatla,23000,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,8220165,"{'Aristocratic': 93, 'Wealthy': 665, 'Comforta...","{'humans': 5842, 'halflings': 17158, 'half-orc...",58,134,Kleptocracy,Exposure to the elements,16,890,135421205
5,Crimmor,Amn,Athkatla,34975,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,10066335,"{'Aristocratic': 286, 'Wealthy': 919, 'Comfort...","{'humans': 34975, 'halflings': 0, 'half-orcs':...",41,236,Patriarchy,Magic mishaps,8,776,135421205
6,Dark Redoubt,Amn,Athkatla,292,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,36500,"{'Aristocratic': 0, 'Wealthy': 1, 'Comfortable...","{'humans': 288, 'halflings': 3, 'half-orcs': 0...",42,0,Militocracy,Exposure to the elements,0,0,135421205
7,Edive,Amn,Athkatla,192,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,39785,"{'Aristocratic': 0, 'Wealthy': 0, 'Comfortable...","{'humans': 192, 'halflings': 0, 'half-orcs': 0...",43,0,Gerontocracy,Lack of medical care,0,0,135421205
8,Esford,Amn,Athkatla,5670,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,2140360,"{'Aristocratic': 2, 'Wealthy': 262, 'Comfortab...","{'humans': 2863, 'halflings': 2806, 'half-orcs...",53,2,Democracy,Disease,0,2,135421205
9,Eshpurta,Amn,Athkatla,23037,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,9796965,"{'Aristocratic': 170, 'Wealthy': 328, 'Comfort...","{'humans': 23037, 'halflings': 0, 'half-orcs':...",41,371,Democracy,Magic mishaps,11,1609,135421205


In [ ]:
# Calculate the total economy of all settlements
total_population = df['settlement_population'].sum()
print(f"The total population of all settlements is {total_population}")


The total population of all settlements is 6527146


In [ ]:
#Define Constants
species = ["humans", "halflings", "half-orcs", "half-drow", "half-elves", "elves", "dwarves", "gnomes", "goblins", "lizardfolk", "yuan-ti", "orcs", "ogres", "asabi", "gnolls",
           "pterafolk", "drow", "centaurs", "wemics", "humanoids"]

# Custom function to calculate average_age with lifestyle modifiers
def calculate_average_age(row):
    try:
        # Initialize variables
        weighted_ages = 0
        total_population = row['settlement_population']
        lifestyle_adjustment = 0

        # Loop over each species in the demographic breakdown
        for species, count in row['demographic_breakdown'].items():
            # Get the age range for this species
            age_range = species_age_ranges.get(species, [1, 100])  # default to human age range if species is not found

            # Calculate the average age for this species
            avg_age_species = sum(age_range) / 2

            # Calculate the weighted age for this species
            weighted_age_species = avg_age_species * (count / total_population)

            # Add to the total weighted ages
            weighted_ages += weighted_age_species

        # Calculate lifestyle adjustment
        for lifestyle, count in row['class_density'].items():
            modifier = lifestyle_modifier.get(lifestyle, 0)
            lifestyle_adjustment += (count / total_population) * modifier

        # Apply lifestyle adjustment to weighted ages
        average_age = weighted_ages + lifestyle_adjustment

        return average_age
    except Exception as e:
        print(f"Error calculating average age for row: {e}")
        return None

# The species_age_ranges dictionary
species_age_ranges = {
    "humans": [1, 100],
    "halflings": [1, 150],
    "half-elves": [1, 180],
    "half-orcs": [1, 75],
    "orcs": [1, 50],
    "elves": [1, 750],
    "dwarves": [1, 350],
    "gnomes": [1, 400]
}

# Lifestyle modifiers
lifestyle_modifier= {
    'Wretched': -5,
    'Squalid': -2,
    'Poor': -1,
    'Modest': 2,
    'Comfortable': 3,
    'Wealthy': 6,
    'Aristocratic': 10
}

# Apply the function across the DataFrame rows
df['average_age'] = df.apply(calculate_average_age, axis=1)
df['average_age'] = df['average_age'].astype(int)

In [ ]:
# Custom function to calculate rumored_treasure_value
def calculate_rumored_treasure_value(row):
    try:
        # Determine the size of the settlement
        size = determine_settlement_size(row['settlement_population'])

        # Calculate the base value by dividing settlement_economy by 1000
        base_value = row['settlement_economy'] / 1000

        # Apply size modifier to the hidden_economy
        size_modifier = {
            'tiny': 0.09,
            'small': 0.09,
            'medium': 0.1,
            'large': .2,
            'mega': .5
        }
        modified_hidden_economy = row['hidden_economy'] * size_modifier.get(size, 1)

        # Add the modified hidden_economy to the base_value
        treasure_value = base_value + modified_hidden_economy

        return treasure_value
    except Exception as e:
        print(f"Error calculating rumored treasure value for row: {e}")
        return None

# Apply the function across the DataFrame rows
idf['rumored_treasure_value'] = idf.apply(calculate_rumored_treasure_value, axis=1)
idf['rumored_treasure_value'] = idf['rumored_treasure_value'].astype(int)


In [ ]:
import random

# List of government types
government_types = ["Autocracy", "Bureaucracy", "Confederacy", "Democracy", "Dictatorship", "Feudalism", "Gerontocracy", "Hierarchy",
                    "Monarchy", "Magocracy", "Militocracy", "Matriarchy", "Oligarchy", "Patriarchy", "Meritocracy", "Plutocracy",
                    "Republic", "Kleptocracy", "Satrapy", "Theocracy"]

# Add a new column "government_type" with random values
df['government_type'] = [random.choice(government_types) for _ in range(len(df))]

# Print the DataFrame to verify the new column
print(df[['region_kingdom', 'government_type']])


      region_kingdom government_type
0                Amn        Monarchy
1                Amn       Autocracy
2                Amn     Militocracy
3                Amn    Gerontocracy
4                Amn       Hierarchy
..               ...             ...
643          Narfell    Dictatorship
644          Narfell       Democracy
645          Narfell       Magocracy
646          Narfell     Bureaucracy
647  The Sword Coast       Magocracy

[648 rows x 2 columns]


In [ ]:
# Define the function to determine settlement size
def determine_settlement_size(population):
    if population < 1000:
        return 'tiny_settlement'
    elif 1000 <= population < 10000:
        return 'small_settlement'
    elif 10000 <= population < 100000:
        return 'medium_settlement'
    elif 100000 <= population < 1000000:
        return 'large_settlement'
    else:
        return 'mega_settlement'

# Define causes of death for each settlement size
causes_of_death = {
    'tiny_settlement': ['Old age', 'Wildlife attacks', 'Starvation', 'Disease', 'Exposure to the elements', 'Accidental falls', 'Lack of medical care', 'Poisonous plants', 'Isolation', 'Magic mishaps'],
    'small_settlement': ['Disease', 'Assassination', 'Poison', 'Old age', 'Starvation', 'Bandit attacks', 'Exposure to the elements', 'Magic mishaps', 'Work-related accidents', 'Wildlife attacks'],
    'medium_settlement': ['Disease', 'Old age', 'Assassination', 'Poison', 'Work-related accidents', 'Starvation', 'Exposure to the elements', 'Bandit attacks', 'Magic mishaps', 'Dueling accidents'],
    'large_settlement': ['Disease', 'Assassination', 'Poison', 'Old age', 'Magic mishaps', 'Work-related accidents', 'Starvation', 'Dueling accidents', 'Wagon Accident', 'Exposure to the elements'],
    'mega_settlement': ['Assassination', 'Disease', 'Poison', 'Magic mishaps', 'Dueling accidents', 'Crime', 'Work-related accidents', 'Starvation', 'Wagon Accident', 'Exposure to the elements']
}

# Function to assign cause of death based on settlement size
def assign_cause_of_death(row):
    size = row['settlement_size']
    return random.choice(causes_of_death[size])

# Add a new column "settlement_size" based on population
df['settlement_size'] = df['settlement_population'].apply(determine_settlement_size)

# Add a new column "cause_of_death" with assigned causes
df['most_likely_cause_of_death'] = df.apply(assign_cause_of_death, axis=1)

# Print the DataFrame to verify the new columns
print(df[['settlement', 'settlement_population', 'settlement_size', 'most_likely_cause_of_death']])


           settlement  settlement_population    settlement_size  \
0            Athkatla                 116163   large_settlement   
1            Amnwater                   7035   small_settlement   
2       Citadel Amnur                    636    tiny_settlement   
3    Citadel Rashturl                    710    tiny_settlement   
4           Coryllvol                  23000  medium_settlement   
..                ...                    ...                ...   
643            N’Jast                     71    tiny_settlement   
644          Peltarch                    313    tiny_settlement   
645           Selmast                    392    tiny_settlement   
646           Snowcap                    495    tiny_settlement   
647            Luskan                  16000  medium_settlement   

    most_likely_cause_of_death  
0                   Starvation  
1                   Starvation  
2             Poisonous plants  
3     Exposure to the elements  
4     Exposure to the elements

In [ ]:
df.head()

,settlement,region_kingdom,capital,settlement_population,area,ruler,military,languages,religions,exports,...,settlement_economy,hidden_economy,class_density,demographic_breakdown,average_age,rumored_treasure_value,region_economy,government_type,settlement_size,most_likely_cause_of_death
0,Athkatla,Amn,Athkatla,116163,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,4271084,380535,"{'Aristocratic': 680, 'Wealthy': 5163, 'Comfor...","{'humans': 116046, 'halflings': 116, 'half-orc...",50,80378,403450,Confederacy,large_settlement,Old age
1,Amnwater,Amn,Athkatla,7035,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,3301,2,"{'Aristocratic': 0, 'Wealthy': 189, 'Comfortab...","{'humans': 2335, 'halflings': 4699, 'half-orcs...",68,3,403450,Matriarchy,small_settlement,Magic mishaps
2,Citadel Amnur,Amn,Athkatla,636,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,17,0,"{'Aristocratic': 0, 'Wealthy': 4, 'Comfortable...","{'humans': 580, 'halflings': 55, 'half-orcs': ...",53,0,403450,Patriarchy,tiny_settlement,Magic mishaps
3,Citadel Rashturl,Amn,Athkatla,710,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,18,0,"{'Aristocratic': 0, 'Wealthy': 5, 'Comfortable...","{'humans': 487, 'halflings': 222, 'half-orcs':...",59,0,403450,Monarchy,tiny_settlement,Old age
4,Coryllvol,Amn,Athkatla,23000,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,66185,685,"{'Aristocratic': 71, 'Wealthy': 626, 'Comforta...","{'humans': 5842, 'halflings': 17158, 'half-orc...",69,134,403450,Kleptocracy,medium_settlement,Work-related accidents


In [ ]:
import pandas as pd
import ast
import random

# Define weights for factors (you can adjust these)
weights = {
    'wealth_distribution': 0.4,
    'trade_balance': 0.2,
    'hidden_economy': 0.1,
    'average_age': 0.3
}

# Define a function to parse the 'class_density' column into dictionaries
def parse_class_density(x):
    try:
        return ast.literal_eval(x)
    except (SyntaxError, ValueError):
        return {}

# Parse the 'class_density' column into dictionaries
idf['class_density'] = idf['class_density'].apply(parse_class_density)

# Now you can access dictionary values in the 'class_density' column
idf['normalized_wealth'] = (idf['class_density'].apply(lambda x: x.get('Aristocratic', 0)) + idf['class_density'].apply(lambda x: x.get('Wealthy', 0))) / (idf['class_density'].apply(lambda x: x.get('Squalid', 0)) + idf['class_density'].apply(lambda x: x.get('Wretched', 0)) + 1)

# Calculate the base inequality score
idf['inequality_score'] = (
    weights['wealth_distribution'] * idf['normalized_wealth'] +
    weights['trade_balance'] * idf['normalized_trade_balance'] +  # Use the 'normalized_trade_balance' column
    weights['hidden_economy'] * idf['normalized_hidden_economy'] +
    weights['average_age'] * idf['normalized_age']
)

# Add randomization to the inequality score within a 10% range
random_factor = random.uniform(0.95, 1.05)  # Random factor between 0.95 and 1.05 (10% range)
idf['inequality_score'] = idf['inequality_score'] * random_factor


In [ ]:
idf['normalized_trade_balance'].tail(100)

548   -0.142857
549   -0.142857
550   -0.142857
551   -0.142857
552   -0.142857
         ...   
643   -0.071429
644   -0.071429
645   -0.071429
646   -0.071429
647    0.071429
Name: normalized_trade_balance, Length: 100, dtype: float64

In [ ]:
import pandas as pd
import numpy as np

# Define the calculate_tax_rate function
def calculate_tax_rate(row):
    # Check if 'imports' and 'exports' are not NaN and are of type list
    if pd.notna(row['imports']) and isinstance(row['imports'], list):
        region_imports = len(row['imports'])
    else:
        region_imports = 0

    if pd.notna(row['exports']) and isinstance(row['exports'], list):
        region_exports = len(row['exports'])
    else:
        region_exports = 0

    # Check if there are aristocrats in the settlement
    if row['class_density'].get('Aristocratic', 0) > 0:
        region_aristocrats = df[df['region_kingdom'] == row['region_kingdom']]['class_density'].apply(lambda x: x.get('Aristocratic', 0)).sum()

        # Check if the region has 100 or more aristocrats
        if region_aristocrats >= 100:
            # Check if there is a military in the region's capital
            if row['capital'] == row['settlement']:
                if pd.notna(row['military']):
                    tax_rate = np.random.randint(10, 21)  # Tax rate between 10% and 20% with military
                else:
                    tax_rate = np.random.randint(1, 11)  # Tax rate between 1% and 10% without military
            else:
                tax_rate = np.random.randint(1, 21)  # Tax rate between 1% and 20% in regions with aristocrats and >=100 total aristocrats
        else:
            tax_rate = 0  # Tax rate is 0 if region has aristocrats but <100 total aristocrats
    else:
        tax_rate = 0  # Tax rate is 0 if no aristocrats in the settlement

    # Adjust tax rate based on total imports and exports in the region
    # (Note: You can use region_imports and region_exports here)

    # Calculate the multiplier based on imports and exports
    multiplier = 1 + (region_imports + region_exports) / 1000

    # Apply the multiplier to the tax rate
    adjusted_tax_rate = tax_rate * multiplier

    return adjusted_tax_rate

# Apply the calculate_tax_rate function to create the tax_rate column
df['tax_rate'] = df.apply(calculate_tax_rate, axis=1)


In [ ]:
# Group by 'region_kingdom' and sum the 'inequality_score' for each region
region_inequality_sum = idf.groupby('region_kingdom')['inequality_score'].sum()

# Sort the regions by the summed inequality score in descending order and take the top 10
top_10_inequal_regions = region_inequality_sum.sort_values(ascending=False).head(10)

# Print the top 10 most unequal regions
print("Top 10 Most Unequal Regions:")
print(top_10_inequal_regions)


Top 10 Most Unequal Regions:
region_kingdom
Impiltur              2777.739202
Damara                1455.685075
Amn                   1264.402661
Aglarond              1179.136733
Moonshae              1126.633245
The Sword Coast        932.575754
Lantan                 913.314460
Cormyr                 853.371977
Tethyr                 478.908764
The Silver Marches     109.643222
Name: inequality_score, dtype: float64


In [ ]:
# Define a function to adjust average_age based on inequality_score
def adjust_average_age(row):
    # Define a scaling factor to control the adjustment
    scaling_factor = 50  # You can adjust this value as needed

    # Calculate the adjusted average age
    adjusted_age = row['average_age'] - scaling_factor * row['inequality_score']

    # Ensure the adjusted age is within a reasonable range
    return max(adjusted_age, 1)  # Minimum age set to 1

# Apply the adjustment function to the DataFrame
idf['adjusted_average_age'] = idf.apply(adjust_average_age, axis=1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 648 entries, 0 to 647
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   settlement                 648 non-null    object 
 1   region_kingdom             648 non-null    object 
 2   capital                    648 non-null    object 
 3   settlement_population      648 non-null    int64  
 4   area                       365 non-null    float64
 5   military                   622 non-null    object 
 6   population_breakdown       648 non-null    object 
 7   region_population          648 non-null    int64  
 8   settlement_economy         648 non-null    int64  
 9   class_density              648 non-null    object 
 10  demographic_breakdown      648 non-null    object 
 11  rumored_treasure_value     648 non-null    int64  
 12  tax_rate                   648 non-null    int64  
 13  settlement_size            648 non-null    object 

In [ ]:

df.head()

,settlement,region_kingdom,capital,settlement_population,area,ruler,military,languages,religions,exports,...,region_population,settlement_economy,class_density,demographic_breakdown,average_age,rumored_treasure_value,region_economy,government_type,most_likely_cause_of_death,tax_rate
0,Athkatla,Amn,Athkatla,116163,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,2920094,4271084,"{'Aristocratic': 680, 'Wealthy': 5163, 'Comfor...","{'humans': 116046, 'halflings': 116, 'half-orc...",39,80378,403450,Confederacy,Old age,19
1,Amnwater,Amn,Athkatla,7035,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,2920094,3301,"{'Aristocratic': 0, 'Wealthy': 189, 'Comfortab...","{'humans': 2335, 'halflings': 4699, 'half-orcs...",55,3,403450,Matriarchy,Magic mishaps,0
2,Citadel Amnur,Amn,Athkatla,636,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,2920094,17,"{'Aristocratic': 0, 'Wealthy': 4, 'Comfortable...","{'humans': 580, 'halflings': 55, 'half-orcs': ...",43,0,403450,Patriarchy,Magic mishaps,0
3,Citadel Rashturl,Amn,Athkatla,710,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,2920094,18,"{'Aristocratic': 0, 'Wealthy': 5, 'Comfortable...","{'humans': 487, 'halflings': 222, 'half-orcs':...",49,0,403450,Monarchy,Old age,0
4,Coryllvol,Amn,Athkatla,23000,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,2920094,66185,"{'Aristocratic': 71, 'Wealthy': 626, 'Comforta...","{'humans': 5842, 'halflings': 17158, 'half-orc...",58,134,403450,Kleptocracy,Work-related accidents,16


In [ ]:
region_climate = {
    'Amn': 'Temperate',  # Coastal and fertile
    'Calimshan': 'Arid',  # Desert and semi-arid regions
    'Chessenta': 'Temperate',  # Mediterranean climate
    'Cormyr': 'Temperate',  # Forested and fertile
    'Damara': 'Cold',  # Northern, colder climate
    'Halruaa': 'Temperate',  # Known for magical barriers, likely moderate climate
    'Impiltur': 'Temperate',  # Coastal with some mountainous terrain
    'Moonshae': 'Jungle/Marsh',  # Isles with varying climates, generally temperate
    'Nimbral': 'Temperate',  # Island with moderate climate
    'Sembia': 'Temperate',  # Trading nation with fertile lands
    'Sossal': 'Cold',  # Far to the northeast, likely very cold
    'Tethyr': 'Temperate',  # Coastal and forested
    'Thay': 'Arid',  # Plateau with harsh climate
    'Thesk': 'Temperate',  # Known for trade, likely moderate climate
    'Turmish': 'Temperate',  # Coastal and forested
    'Unther': 'Arid',  # Desert-like with ancient ruins
    'The Sword Coast': 'Temperate',  # Varying climates, mostly temperate
    'Aglarond': 'Temperate',  # Forested peninsula
    'Evereska': 'Temperate',  # Elven city-state in a valley
    'Evermeet': 'Temperate',  # Elven island, magically preserved
    'Gundarlun': 'Cold',  # Northern islands
    'Ruathym': 'Cold',  # Island with a harsh, cold climate
    'Trisk (Kingdom of the Purple Rocks)': 'Temperate',  # Island, uncertain climate
    'Tuern': 'Cold',  # Volcanic island to the north
    'Hartsvale': 'Cold',  # Located in a valley within mountains
    'Lantan': 'Temperate',  # Island nation known for technology
    'Lapaliiya': 'Jungle/Marsh',  # Coastal with some jungles
    'Luiren': 'Temperate',  # Homeland of the halflings, likely moderate climate
    'The Silver Marches': 'Cold',  # Northern frontier with colder climate
    'Dwarfholds of the North': 'Cold',  # Mountainous and northern
    'Orlumbor': 'Temperate',  # Wooded island
    'Mulhorand': 'Arid',  # Desert with some fertile river valleys
    'Narfell': 'Cold',  # Plains with harsh winters
}
climate_temperature_range = {
    'Arid': [65, 100],  # Temperature range in Fahrenheit
    'Cold': [0, 40],
    'Temperate': [45, 70],
    'Jungle/Marsh': [65, 90]
}

average_rainfall = {
    'Arid': [0, 10],  # Rainfall range in inches per year
    'Cold': [20, 40],
    'Temperate': [30, 50],
    'Jungle/Marsh': [45, 65]
}
import random

def assign_climate_attributes(row):
    try:
        # Extract climate from region
        climate = region_climate.get(row['region_kingdom'], 'Temperate')  # Default to 'Temperate' if region is not found

        # Get temperature and rainfall ranges for the climate
        temp_range = climate_temperature_range[climate]
        rainfall_range = average_rainfall[climate]

        # Randomly assign values within the ranges
        average_temp = random.uniform(temp_range[0], temp_range[1])
        annual_rainfall = random.uniform(rainfall_range[0], rainfall_range[1])

        return average_temp, annual_rainfall

    except Exception as e:
        print(f"Error assigning climate attributes for row: {e}")
        return None

# Apply the function to the DataFrame
df['average_temperature'], df['annual_rainfall'] = zip(*df.apply(assign_climate_attributes, axis=1))


In [ ]:
df.head()

,settlement,region_kingdom,capital,settlement_population,area,ruler,military,languages,religions,exports,...,class_density,demographic_breakdown,average_age,rumored_treasure_value,government_type,most_likely_cause_of_death,tax_rate,region_economy,average_temperature,annual_rainfall
0,Athkatla,Amn,Athkatla,116163,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,"{'Aristocratic': 15, 'Wealthy': 3644, 'Comfort...","{'humans': 116046, 'halflings': 116, 'half-orc...",39,858487,Confederacy,Starvation,12.0,135421205,58.016133,36.582132
1,Amnwater,Amn,Athkatla,7035,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,"{'Aristocratic': 1, 'Wealthy': 122, 'Comfortab...","{'humans': 2335, 'halflings': 4699, 'half-orcs...",55,300,Matriarchy,Starvation,16.0,135421205,67.568673,38.175684
2,Citadel Amnur,Amn,Athkatla,636,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,"{'Aristocratic': 0, 'Wealthy': 3, 'Comfortable...","{'humans': 580, 'halflings': 55, 'half-orcs': ...",43,1,Patriarchy,Poisonous plants,0.0,135421205,69.149239,44.558120
3,Citadel Rashturl,Amn,Athkatla,710,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,"{'Aristocratic': 0, 'Wealthy': 6, 'Comfortable...","{'humans': 487, 'halflings': 222, 'half-orcs':...",49,1,Monarchy,Exposure to the elements,0.0,135421205,52.529309,49.359230
4,Coryllvol,Amn,Athkatla,23000,NaN,The Council of Six,Various,"Chondathan, Common","Bane, Chauntea, Cyric, Selûne, Sune, Waukeen","[""['ale'"", ""'amour'"", ""'beer'"", ""'caravan item...",...,"{'Aristocratic': 93, 'Wealthy': 665, 'Comforta...","{'humans': 5842, 'halflings': 17158, 'half-orc...",58,6684,Kleptocracy,Exposure to the elements,12.0,135421205,57.109727,43.269797


In [ ]:
    data = {
            "Wizarding School": np.random.choice(["Yes", "No"]),
            "Dragon Sightings": np.random.randint(0, 50),
            "Tax Rate": np.random.randint(0, 100),
            "Number of Landowners": np.random.randint(1, 200),
            "Number of Homeowners": np.random.randint(10, 10000),
            "Average Rent": np.random.randint(1, 5000),
            "Average Debt by Household": np.random.randint(0, 100000),
            "Number of Active Adventurers": np.random.randint(0, 200),
            "Number of Active Wizards": np.random.randint(0, 20),
            "Likelihood of Getting Cursed": np.random.randint(0, 10),
            "Danger Level": np.random.randint(0, 10),
            "Openness to Strangers": np.random.randint(0, 100),
            "Political Climate": np.random.randint(0, 100),
            "Magical Climate": np.random.randint(0, 10),
            "Fate Ranking": np.random.randint(0, 10),
        }


No cities available to distribute population.
No cities available to distribute population.


TypeError: ignored

In [ ]:
wizard_schools = {
    'Arrabar': 'Academia Vilhonus',
    'Chondath': 'Academia Vilhonus',
    'Thay': 'Academy of Shapers and Binders',
    ' Waterdeep': 'Academy of Stargazers',
    'Mintar': 'Academy of the Drawn Sword',
    ' Lake of Steam': 'Academy of the Drawn Sword',
    'Seventon': "Alimon's Mental Academy",
    'Netheril': "Alimon's Mental Academy",
    'Sundabar': 'Anstruth',
    'Gheldaneth': 'Arcanum of Magic',
    'Mulhorand': 'Arcanum of Magic',
    'Silverymoon': "Arkhen's Invocatorium",
    'Huzuz': "Al-Bidir Sallah (Home of the Seekers of Knowledge)",
    'Zakhara': "Al-Bidir Sallah (Home of the Seekers of Knowledge)",
    'Waterdeep': 'Blackstaff Academy',
    'Luskan': 'Blackstaff Academy',
    'Baldurs Gate': "Briel's School of the Arcane",
    'The Sword Coast': "Briels School of the Arcane",
    'Zazesspur': 'Canaith',
    'Tethyr': 'Canaith',
    'Evereska': 'College of Magic and Arms',
    'The Sword Coast Faerûn': 'College of Magic and Arms',
    'Heralds Holdfast': 'College of the Herald',
    'Berdusk': 'College of Doss',
    'Myth Nantar': "Dukars' Academy",
    'Waterdeep': 'Dweomercore',
    'Waterdeep': 'Eltorchul Academy',
    'Loudwater': "Enalin's Shop of Adventurers' Supplies",
    'Delimbiyr Vale': "Enalin's Shop of Adventurers' Supplies",
    'Heliogabalus': 'Ends Rest',
    'Heldapan': 'Gathering of Magicians',
    'Durpar': 'Gathering of Magicians',
    'Akkaido': 'Golden Fox Martial Arts School',
    'Huzuz': 'Grand University of Huzuz',
    'Medina al-Afyal': 'Great Mosque of Selan',
    'Island of Afyal': 'Great Mosque of Selan',
    'Almorel': 'Great School of Deneir',
    'Waterdeep': 'Halasters Heirs',
    'Evereska': 'Hall of the Kaliesherai',
    'Waterdeep': 'House of Healing'
}

def assign_magic_academy(row):
    settlement = row['settlement']
    if settlement in wizard_schools:
        return wizard_schools[settlement]
    else:
        return 'None'  # Fill with 'None' if there's no school

# Assuming you have a DataFrame named df
df['magic_academy'] = df.apply(assign_magic_academy, axis=1)

# To keep track of schools that do not get placed
unplaced_schools = [school for school in wizard_schools.values() if school not in df['magic_academy'].unique()]

# Print unplaced schools
print("Unplaced Schools:", unplaced_schools)


In [ ]:
def add_magic_academy(df):
    def check_population(row):
        if row['settlement_population'] > 15000 and row['magic_academy'] == 'None':
            # If population is over 10000 and academy is 'None', replace it with 'guilds'
            return 'guilds'
        return row['magic_academy']

    df['magic_academy'] = df.apply(lambda row: check_population(row), axis=1)
    return df
df = add_magic_academy(df)

In [ ]:
# Check the count of each unique value in the 'magic_academy' column
academy_counts = df['magic_academy'].value_counts()

# Check if 'guilds' is in the index of unique values
if 'guilds' in academy_counts.index:
    # 'guilds' was added, and you can access its count using academy_counts['guilds']
    guilds_count = academy_counts['guilds']
    print(f"'guilds' was added {guilds_count} times.")
else:
    # 'guilds' was not added
    print("'guilds' was not added.")


'guilds' was added 78 times.


In [ ]:
num_cities = df.shape[0]

species_age_ranges = {
            "Humans": [1, 100],
            "Halflings": [1, 150],
            "Orcs": [1, 50],
            "Elves": [1, 750],
            "Dwarves": [1, 350],
            "Gnomes": [300, 500]
        }

# Adjusting Danger Level
df['Danger Level'] = (
    df['Dragon Sightings']
    + df['Political Climate']
    + df['Magical Climate']
    + df['Likelihood of Getting Cursed']
    + df['Tax Rate']
    + df['Average Debt by Household']
    + df['Rumored Treasure Value']
    + df['Openness to Strangers'].apply(lambda x: 10 if x <= 2.5 or x >= 7.5 else 0)
).astype(int)

# Increase "Danger Level" if Orcs are present
df.loc[df['Percentage of population Orcs'] > 0, 'Danger Level'] += 1

# Decrease "Danger Level" if Elves are present
df.loc[df['Percentage of population Elves'] > 0, 'Danger Level'] -= 1

# Adjusting Danger Level based on the presence of Orcs and Elves
df['Danger Level'] = df.apply(lambda row: np.random.randint(1, 11)
                                          + (3 if row['Percentage of population Orcs'] > 0 else 0)
                                          - (2 if row['Percentage of population Elves'] > 0 else 0), axis=1)

# Adjusting Openness to Strangers
df['Openness to Strangers'] = df.apply(lambda row: np.random.randint(0, 11)
                                                   - (3 if row['Percentage of population Elves'] > 0
                                                       or row['Percentage of population Orcs'] > 0 else 0), axis=1)

# Adjusting Magical Climate
df['Magical Climate'] = df.apply(lambda row: row['Magical Climate']
                                             + (row['Percentage of population Elves'] / 100 * 2)
                                             + (row['Dragon Sightings'] / 10)
                                             + row['Number of Active Wizards'], axis=1)

# Pre-calculation of sum values
grouped = df.groupby('Region')
region_gold_mapping = grouped[['Active Economy (Gold)', 'Hidden Economy (Gold)']].sum(numeric_only=True).sum(axis=1)
df['Total Gold in Region'] = df['Region'].map(region_gold_mapping)

# Main DataFrame Manipulations
df['Dragon Sightings'] = df.apply(calculate_dragon_sightings, axis=1)
df['Dragon Sightings'] = ((df['Dragon Sightings'] - df['Dragon Sightings'].min()) /
                          (df['Dragon Sightings'].max() - df['Dragon Sightings'].min()) * 50).astype(int)

df['Average Age of Population'] = df.apply(lambda row: calculate_average_age(row, species_age_ranges), axis=1)
df['Average Age of Population'] -= df['Danger Level'] * 0.5
df['Average Age of Population'] = df['Average Age of Population'].clip(lower=1)

df['Hidden Economy (Gold)'] = df['Number of Nobility'] * 1e4
df['Openness to Strangers'] = df.apply(calculate_openness_to_strangers, axis=1)

# Adjusting population distribution for specific species across cities
def adjust_population_distribution(df, species, percentage, city_count):
    total_population = df['Population of City'].sum()
    target_population = int(total_population * (percentage / 100))
    selected_cities = df.sample(city_count)
    each_city_population = target_population // city_count
    for i, city in selected_cities.iterrows():
        df.at[i, f'Percentage of population {species}'] = (each_city_population / city['Population of City']) * 100

# Adjusting Economy Distribution
total_active_economy = np.sum(df['Active Economy (Gold)'])
top_10_cities_active_economy = total_active_economy * 0.50
next_100_cities_active_economy = total_active_economy * 0.25
remaining_cities_active_economy = total_active_economy * 0.25

# Known Treasure and Rumored Treasure Calculations
df['Known Treasure'] = df['Active Economy (Gold)'] * 0.50
df['Rumored Treasure'] = (df['Active Economy (Gold)'] * 0.50) + df['Hidden Economy (Gold)']

# Adjusting Political Climate and Fate Ranking
df['Wizarding School'] = np.random.choice(['yes', 'no'], num_cities)
df['Number of Homeowners'] = (df['Population of City'] * np.random.uniform(0.5, 0.7)).astype(int)
df['Average Rent'] = np.random.uniform(500, 2000, num_cities)
df['Average Age of Population'] = np.random.uniform(25, 70, num_cities)
df['Known Treasure Value'] = np.random.uniform(0, 1e6, num_cities)

# Adjust the tax rate based on the number of nobility
df['Tax Rate'] = df.apply(lambda row: 0 if row['Number of Nobility'] == 0 else row['Tax Rate'], axis=1)

# (You need to define 'Average Debt by Household' and 'Rumored Treasure Value' before using them in the next formula)
df['Political Climate'] = (df['Tax Rate'] + df['Number of Homeowners']/df['Population of City'] +
                              df['Average Rent']/2000 + df['Average Debt by Household']/1e4 +
                              df['Average Age of Population']/70 + df['Hidden Economy (Gold)']/1e6 +
                              df['Known Treasure Value']/1e6 + df['Rumored Treasure Value']/1e6) / 8

df['Fate ranking'] = (df['Rumored Treasure Value'] / 1e6) + np.where(df['Wizarding School'] == 'yes', 1, 0) + df['Political Climate']

In [ ]:
df.head(50)

,City,Region,Nation,Government Type,Population of City,Typical Cause of Death,Wizarding School,Known Treasure Value,Rumored Treasure Value,Dragon Sightings,...,Percentage of population Dwarves,Number of Dwarves,Percentage of population Gnomes,Number of Gnomes,Active Economy (Gold),Hidden Economy (Gold),Average Age of Population,Known Treasure,Rumored Treasure,Fate ranking
234,Abriymoch,Region1,Nation1,Democracy,150953,Conflict,no,442269.743103,4292239,5,...,0.06,90.5718,5.410000,8166.5573,5.619318e+01,48460000.0,60.925487,2.809659e+01,4.846003e+07,15.555876
176,Aelor,Region1,Nation1,Monarchy,89041,Old Age,no,476091.389884,6870747,9,...,1.13,1006.1633,35.350000,31475.9935,8.362205e+01,2280000.0,59.981192,4.181103e+01,2.280042e+06,14.640789
215,Ageadren,Region1,Nation2,Theocracy,316319,Disease,yes,622975.875787,9325648,4,...,29.73,94041.6387,2.220000,7022.2818,7.905514e+01,24680000.0,45.851169,3.952757e+01,2.468004e+07,23.450614
312,Airspur,Region1,Nation1,Democracy,168364,Disease,yes,583496.534964,4625153,0,...,4.45,7492.1980,14.240000,23975.0336,9.781790e+01,25250000.0,63.902849,4.890895e+01,2.525005e+07,19.532837
214,Akanax,Region1,Nation2,Theocracy,62586,Disease,yes,374263.381648,5253459,9,...,6.57,4111.9002,10.020000,6271.1172,5.391364e+01,41070000.0,66.680443,2.695682e+01,4.107003e+07,15.588609
161,Akkaido,Region1,Nation2,Democracy,49444,Old Age,no,884114.385300,1653817,4,...,31.87,15757.8028,46.850000,23164.5140,7.530065e+01,13910000.0,39.775496,3.765033e+01,1.391004e+07,15.737580
334,Al-Anwahr,Region2,Nation1,Democracy,18381,Conflict,yes,667660.523127,9956084,21,...,0.59,108.4479,8.860000,1628.5566,8.338734e+01,28130000.0,35.111252,4.169367e+01,2.813004e+07,26.704809
17,Alashan,Region1,Nation1,Monarchy,23634,Accident,yes,205406.330600,1635605,6,...,19.19,4535.3646,6.650000,1571.6610,9.976553e+01,16460000.0,45.891836,4.988276e+01,1.646005e+07,16.338954
376,Almraiven,Region1,Nation2,Oligarchy,30854,Conflict,yes,319902.993506,3115229,3,...,2.90,894.7660,5.770000,1780.2758,9.973902e+01,12910000.0,33.069686,4.986951e+01,1.291005e+07,13.405409
14,Ammathluir,Region2,Nation2,Democracy,340906,Conflict,yes,787878.101703,5719607,22,...,3.57,12170.3442,7.530000,25670.2218,7.536108e+01,47840000.0,64.780130,3.768054e+01,4.784004e+07,21.829522


In [ ]:
# Save the updated DataFrame back to the Excel file
df.to_excel('/content/drive/MyDrive/Colab Notebooks/DnD/fictional_data.xlsx', index=False)

In [ ]:

# Step 4: Calculating Danger Level
df['Dragon Sightings'] = np.random.randint(0, 100, cities)
df['Political Climate'] = np.random.uniform(0, 1, cities)
df['Magical Climate'] = np.random.uniform(0, 1, cities)
df['Openness to Strangers'] = np.random.uniform(0, 1, cities)
df['Average Debt by Household'] = np.random.uniform(0, 1e4, cities)
df['Rumored Treasure Value'] = np.random.uniform(0, 1e6, cities)

bottom_quartile = df['Openness to Strangers'].quantile(0.25)
top_quartile = df['Openness to Strangers'].quantile(0.75)
openness_score = np.where(df['Openness to Strangers'] < bottom_quartile, 1,
                np.where(df['Openness to Strangers'] > top_quartile, 1, 0))

df['Likelihood of Getting Cursed'] = df['Magical Climate'] + openness_score
df['Danger Level'] = (df['Dragon Sightings'] + df['Political Climate'] +
                             df['Magical Climate'] + df['Likelihood of Getting Cursed'] +
                             df['Tax Rate'] + df['Average Debt by Household'] +
                             df['Rumored Treasure Value'] + openness_score) / 8


In [ ]:
# Step 9: Setting Political Climate
df['# Homeowners'] = (df['City Population'] * np.random.uniform(0.5, 0.7)).astype(int)
df['Average Rent'] = np.random.uniform(500, 2000, cities)
df['Average Age of Population'] = np.random.uniform(25, 70, cities)
df['Known Treasure Value'] = np.random.uniform(0, 1e6, cities)

df['Political Climate'] = (df['Tax Rate'] + df['# Homeowners']/df['City Population'] +
                                  df['Average Rent']/2000 + df['Average Debt by Household']/1e4 +
                                  df['Average Age of Population']/70 + df['Hidden Economy (Gold)']/1e6 +
                                  df['Known Treasure Value']/1e6 + df['Rumored Treasure Value']/1e6) / 8

# Step 10: Adjusting Fate Ranking
df['Fate ranking'] = (df['Rumored Treasure Value'] / 1e6) + np.where(df['Wizarding School'] == 'yes', 1, 0) + df['Political Climate']

# Step 11: Making Dataset Generation Replicable
def generate_dataset(seed=42):
    np.random.seed(seed)
    # ... (insert all the previous code that generates the data here)
    return df

# Generate a new dataset with a different seed each time to get a slightly different dataset
new_dataset = generate_dataset(seed=45)
print(new_dataset.head())

# You can use the generate_dataset function to create new datasets with minor variations each time



In [ ]:
df.head()

,City,Region,Nation,Population of City,Government,Percentage of population Humans,Percentage of population Halflings,Percentage of population Orcs,Percentage of population Elves,Percentage of population Dwarves,...,Environmental Climate,Average Temperature,Average Rainfall,Political Climate,Magical Climate,Likelihood of Getting Cursed,Danger Level,Openness to Strangers,Notable Landmarks,Fate ranking
0,Abriymoch,Region2,Nation2,15922,Oligarchy,92.890664,7.904120,71.520153,80.264236,10.024334,...,21.149629,28.056975,51.715684,37.255145,16.583509,36.113485,90.531226,60.926050,Landmark1,45
1,Aelor,Region2,Nation3,12583,Oligarchy,76.311884,1.607618,11.346570,68.282074,80.864470,...,85.989144,42.689376,84.906626,15.165676,92.435707,33.064317,86.386139,5.308632,Landmark3,416
2,Ageadren,Region3,Nation2,84272,Oligarchy,12.111212,54.610246,89.830203,74.545746,30.843258,...,41.153760,21.158781,9.216342,65.065520,44.178035,94.057346,57.030169,46.249247,Landmark3,227
3,Airspur,Region2,Nation3,50224,Oligarchy,85.766203,73.721184,42.047838,30.094798,48.466543,...,47.134302,16.988302,23.385766,41.125886,54.351449,46.155325,35.290176,39.565244,Landmark2,86
4,Airspur (Chessenta),Region1,Nation3,17081,Oligarchy,9.291475,49.656559,84.705312,85.098654,94.914073,...,65.841312,78.579678,66.109504,50.585987,21.530267,12.120318,74.095791,33.795816,Landmark1,519


In [ ]:
# Step 2: Adjust Danger Level

# Convert "Danger Level" to an integer between 1 and 10
df['Danger Level'] = (df['Dragon Sightings'] + df['Political Climate'] + df['Magical Climate'] + df['Likelihood of Getting Cursed'] + df['Tax Rate'] + df['Average Debt by Household'] + df['Rumored Treasure Value'] + df['Openness to Strangers'].apply(lambda x: 10 if x <= 2.5 or x >= 7.5 else 0)).astype(int)

# We normalize the "Danger Level" to be between 1 and 10
df['Danger Level'] = ((df['Danger Level'] - df['Danger Level'].min()) / (df['Danger Level'].max() - df['Danger Level'].min()) * 9 + 1).astype(int)

# Increase "Danger Level" if Orcs are present
df.loc[df['Percentage of population Orcs'] > 0, 'Danger Level'] += 1

# Decrease "Danger Level" if Elves are present
df.loc[df['Percentage of population Elves'] > 0, 'Danger Level'] -= 1

# Ensure "Danger Level" remains within the range 1-10
df['Danger Level'] = df['Danger Level'].clip(1, 10)


KeyError: ignored

In [ ]:
# Step 3: Modify Dragon Sightings

# First, we'll create a mapping of the total gold per region
region_gold_mapping = df.groupby('Region')['Active Economy (Gold)', 'Hidden Economy (Gold)'].sum().sum(axis=1)

# Next, we'll adjust the Dragon Sightings based on the total gold in the region and the Fate ranking of the city
df['Dragon Sightings'] = df.apply(lambda row: (row['Fate ranking'] * (region_gold_mapping[row['Region']] / 1e6)), axis=1)

# We'll then normalize this to be a number between 0 and 50
df['Dragon Sightings'] = ((df['Dragon Sightings'] - df['Dragon Sightings'].min()) / (df['Dragon Sightings'].max() - df['Dragon Sightings'].min()) * 50).astype(int)

# Making sure "Dragon Sightings" does not exceed 50
df['Dragon Sightings'] = df['Dragon Sightings'].clip(0, 50)


KeyError: ignored

In [ ]:
import numpy as np

# Adjusting population distribution for specific species across cities
def adjust_population_distribution(df, species, percentage, city_count):
    total_population = df['Population of City'].sum()
    target_population = int(total_population * (percentage / 100))
    selected_cities = df.sample(city_count)
    each_city_population = target_population // city_count
    for i, city in selected_cities.iterrows():
        df.at[i, f'Percentage of population {species}'] = (each_city_population / city['Population of City']) * 100

# Adjusting the population distribution
adjust_population_distribution(df, "Gnomes", 99, 2)
adjust_population_distribution(df, "Dwarves", 80, 4)
adjust_population_distribution(df, "Elves", 95, 5)
adjust_population_distribution(df, "Orcs", 99, 1)

# Adjusting Dragon Sightings
df['Dragon Sightings'] = (df['Region'].map(df.groupby('Region')['Fate ranking'].sum() + df.groupby('Region')['Active Economy (Gold)'].sum()) / 1e6).clip(0, 50)

# Adjusting Danger Level
df['Danger Level'] = df.apply(lambda row: np.random.randint(1, 11) + (3 if row['Percentage of population Orcs'] > 0 else 0) - (2 if row['Percentage of population Elves'] > 0 else 0), axis=1)

# Adjusting Openness to Strangers
df['Openness to Strangers'] = df.apply(lambda row: np.random.randint(0, 11) - (3 if row['Percentage of population Elves'] > 0 or row['Percentage of population Orcs'] > 0 else 0), axis=1)

# Adjusting Magical Climate
df['Magical Climate'] = df.apply(lambda row: row['Magical Climate'] + (row['Percentage of population Elves'] / 100 * 2) + (row['Dragon Sightings'] / 10) + row['Number of Active Wizards'], axis=1)

# Adjusting Hidden Economy based on the number of nobility
df['Hidden Economy (Gold)'] = df.apply(lambda row: row['Hidden Economy (Gold)'] * (1 + (row['# of Nobility'] / row['Population of City']) * 100), axis=1)

# Ensuring Percentage values are rounded to 2 decimal places
percentage_columns = [col for col in df.columns if 'Percentage' in col]
df[percentage_columns] = df[percentage_columns].round(2)

# Making sure Openness to Strangers is between 0 and 10
df['Openness to Strangers'] = df['Openness to Strangers'].clip(0, 10)


KeyError: ignored

In [ ]:
def calculate_dragon_sightings(row):
    # The dragon sightings are influenced by the total gold in a region and the fate ranking of the city
    region_gold = df[df['Region'] == row['Region']]['Active Economy (Gold)'].sum() + df[df['Region'] == row['Region']]['Hidden Economy (Gold)'].sum()
    max_sightings = min(50, row['Fate ranking'] + region_gold // 1e6)
    return np.random.randint(0, max_sightings+1)

def adjust_danger_level(row):
    # Adjusting danger level based on the presence of Orcs and Elves
    danger_level = row['Danger Level']
    if row['Percentage of population Orcs'] > 0:
        danger_level += 1
    if row['Percentage of population Elves'] > 0:
        danger_level -= 1
    return min(max(1, danger_level), 10)

def calculate_magical_climate(row):
    # Adjusting magical climate based on dragon sightings and the number of active wizards
    return row['Dragon Sightings'] * 0.1 + row['Number of Active Wizards'] * 0.5

def calculate_openness_to_strangers(row):
    # Adjusting openness to strangers based on the presence of Elves or Orcs
    openness = row['Openness to Strangers']
    if row['Percentage of population Elves'] > 0 or row['Percentage of population Orcs'] > 0:
        openness -= 2
    return min(max(0, openness), 10)

# Applying the functions
df['Dragon Sightings'] = df.apply(calculate_dragon_sightings, axis=1)
df['Danger Level'] = df.apply(adjust_danger_level, axis=1)
df['Magical Climate'] = df.apply(calculate_magical_climate, axis=1)
df['Openness to Strangers'] = df.apply(calculate_openness_to_strangers, axis=1)


KeyError: ignored

In [ ]:
# Step 9.1: Calculate Dragon Sightings based on total gold in a region and the Fate ranking of the city
df['Dragon Sightings'] = (df['Total Gold in Region'] / 1e6) + df['Fate ranking']
df['Dragon Sightings'] = df['Dragon Sightings'].apply(lambda x: min(x, 50))

# Step 9.2: Modify Danger Level
def calculate_danger_level(row):
    danger_level = row['Danger Level']
    if row['Percentage of population Orcs'] > 0:
        danger_level += 1
    if row['Percentage of population Elves'] > 0:
        danger_level -= 1
    return min(max(int(danger_level), 1), 10)
df['Danger Level'] = df.apply(calculate_danger_level, axis=1)

# Step 9.3: Modify Magical Climate
def calculate_magical_climate(row):
    magical_climate = row['Magical Climate']
    if row['Percentage of population Elves'] > 0:
        magical_climate += 1
    magical_climate += row['Dragon Sightings'] / 10
    magical_climate += row['Number of Active Wizards'] / 2
    return min(max(int(magical_climate), 1), 10)
df['Magical Climate'] = df.apply(calculate_magical_climate, axis=1)

# Step 9.4: Modify Openness to Strangers
def calculate_openness_to_strangers(row):
    openness = row['Openness to Strangers']
    if row['Percentage of population Elves'] > 0 or row['Percentage of population Orcs'] > 0:
        openness -= 2
    return min(max(int(openness), 0), 10)
df['Openness to Strangers'] = df.apply(calculate_openness_to_strangers, axis=1)

# Step 9.5: Modify Hidden Economy
df['Hidden Economy'] = df['# of Nobility'] * 1e4
